In [ ]:
%cd drive/My Drive/learning/data_science/coding_the_matrix/resources/

[Errno 2] No such file or directory: 'drive/My Drive/learning/data_science/coding_the_matrix/resources/'
/content/drive/My Drive/learning/data_science/coding_the_matrix/resources


In [ ]:
# Copyright 2013 Philip N. Klein
"""
This file contains a simple plotting interface, which uses an Iframe to
present a plot of points represented as either complex numbers or 2-vectors.
"""
from numbers import Number
from IPython.display import IFrame,HTML, display
import tempfile
import os
import atexit
from io import StringIO


def plot(L, scale=4, dot_size = 3):
    """ plot takes a list of points, optionally a scale (relative to a 200x200 frame),
        optionally a dot size (diameter) in pixels.It produces a string with 
        SVG representing the given plot, and opens the file in an Iframe. It returns nothing.
    """
    scalar = 200./scale
    origin = (210, 210)
    out_str = StringIO()
    out_str.write(
            """<html>
            <head>
            <title>plot</title>
            </head>
            <body>
            <svg height="1000" width="1000" xmlns="http://www.w3.org/2000/svg">\n'
            <line x1="0" y1="210" x2="420" y2="210"
            style="stroke:rgb(0,0,0);stroke-width:2"/>
            <line x1="210" y1="0" x2="210" y2="420"
            style="stroke:rgb(0,0,0);stroke-width:2"/>""")
    parsedImgPts = list()
    for pt in L:
      if isinstance(pt, Number):
        x,y = pt.real, pt.imag
      else:
          if isinstance(pt, tuple) or isinstance(pt, list):
            x,y = pt
          else:
            raise ValueError
      parsedImgPts.append((origin[0]+scalar*x, origin[1]-scalar*y)) 
    for (a, b) in parsedImgPts:       
        out_str.write('<circle cx="%d" cy="%d" r="%d" fill="red"/>\n'
                          % (a,b , dot_size))
    out_str.write("""</svg>\n</body>\n</html>""")
    display(HTML(out_str.getvalue()))
    out_str.close()
        

In [ ]:
L = [(2,2), (3,2), (1.75,1), (2,1), (2.25,1), (2.5,1), (2.75,1), (3,1), (3.25,1)]
plot(L)

In [ ]:
from numbers import Number
from IPython.display import IFrame,HTML, display
display(HTML("""<html>
            <head>
            <title>plot</title>
            </head>
            <body>
            <svg height="1000" width="1000" xmlns="http://www.w3.org/2000/svg">
            <line x1="0" y1="210" x2="420" y2="210"
            style="stroke:rgb(0,0,0);stroke-width:2"/>
            <line x1="210" y1="0" x2="210" y2="420"
            style="stroke:rgb(0,0,0);stroke-width:2"/><circle cx="347" cy="160" r="4" fill="red"/>
<circle cx="360" cy="110" r="4" fill="red"/>
<circle cx="297" cy="160" r="4" fill="red"/>
<circle cx="360" cy="160" r="4" fill="red"/>
<circle cx="335" cy="160" r="4" fill="red"/>
<circle cx="310" cy="110" r="4" fill="red"/>
<circle cx="372" cy="160" r="4" fill="red"/>
<circle cx="322" cy="160" r="4" fill="red"/>
<circle cx="310" cy="160" r="4" fill="red"/>
</svg>
</body>
</html>"""))